In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as seabornInstance
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import RUSBoostClassifier



from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn import preprocessing
from pathlib import Path
from sklearn import datasets, metrics, model_selection, svm
from sklearn.utils import resample
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.ensemble.base module is  deprecated in version 0.22 and will be removed in

In [2]:
data = pd.read_csv(r"X:\Lab Members\Sung\SYSHIN0228\RERC_STARS_Sensor_Analysis\Prediction_Admission_AIR_AoM_Analysis\Prediction_Machine_Learning\Feature_Matrix_Prediction_6MWT.csv", sep=',')
data = data.dropna()

CM = np.zeros((2,2))
W_F1 = np.zeros((10,100))

CoT = 0
for ML_model in range(3):
    for n in [10, 20, 30, 60, 90, 120, 180, 240, 300, 360]:
        df = data.query('cut_off_time == @n')
        # print(df)
        # df.head()

        # Type
        k = 0
        for i in df['type']:
            if i == 'hemorrhage':
                df['type'].values[k] = 0
            elif i == 'ischemic':
                df['type'].values[k] = 1
            elif i == 'both':
                df['type'].values[k] = 2 
            else:
                df['type'].values[k] = 3   
            k+=1

        # Lesion_location
        k = 0
        for i in df['Lesion_location']:
            if i == 'R':
                df['Lesion_location'].values[k] = 0
            elif i == 'L':
                df['Lesion_location'].values[k] = 1
            elif i == 'bilateral':
                df['Lesion_location'].values[k] = 2 
            else:
                df['Lesion_location'].values[k] = 3   
            k+=1

        # Hemiparesis
        k = 0
        for i in df['Hemiparesis']:
            if i == 'R':
                df['Hemiparesis'].values[k] = 0
            elif i == 'L':
                df['Hemiparesis'].values[k] = 1
            else:
                df['Hemiparesis'].values[k] = 2  
            k+=1

        # Hemiparesis
        k = 0
        for i in df['lifestyle']:
            if i == 'Highly-active':
                df['lifestyle'].values[k] = 0
            elif i == 'Highly-Active':
                df['lifestyle'].values[k] = 0
            elif i == 'Moderately':
                df['lifestyle'].values[k] = 1
            else: # Sedentary
                df['lifestyle'].values[k] = 2  
            k+=1

        # print(df)

        # Feature input
        # Demo + CO + Sensor data
        df_feature = df[[
            # Demographics
            'Age', 'SexM0F1', 'Height', 'Weight',
            'type', 'timesinceCVA', 'Lesion_location', 'Hemiparesis', 'lifestyle',
            # Clinical outcomes at admission
#             'FIM_AD', 'BBS_AD', 'MWT10_SSV_AD', 'MWT6_AD', 'TUG_AD',
            # Amount of motion
            'AoM_Pel_norm', 'AoM_Ankle_US_norm', 'AoM_Ankle_AS_norm',
            # Signal characteristics of Gyroscope
            'SC_Gyr_norm_mean', 'DLS_R_Gyr_norm_mean', 'DLS_L_Gyr_norm_mean',
            'SC_Gyr_norm_range', 'DLS_R_Gyr_norm_range', 'DLS_L_Gyr_norm_range',
            'SC_Gyr_norm_rms', 'DLS_R_Gyr_norm_rms', 'DLS_L_Gyr_norm_rms',
            'SC_Gyr_norm_std', 'DLS_R_Gyr_norm_std', 'DLS_L_Gyr_norm_std',
            'SC_Gyr_norm_skew', 'DLS_R_Gyr_norm_skew', 'DLS_L_Gyr_norm_skew',
            'SC_Gyr_norm_kurtosis', 'DLS_R_Gyr_norm_kurtosis', 'DLS_L_Gyr_norm_kurtosis',
            'SC_Gyr_norm_SamEn', 'DLS_R_Gyr_norm_SamEn', 'DLS_L_Gyr_norm_SamEn',
            'SC_Gyr_norm_DAmp', 'DLS_R_Gyr_norm_DAmp', 'DLS_L_Gyr_norm_DAmp',
            'SC_Gyr_norm_DFreq', 'DLS_R_Gyr_norm_DFreq', 'DLS_L_Gyr_norm_DFreq',
            # Signal characteristics of acceleration
            'SC_Acc_norm_mean', 'DLS_R_Acc_norm_mean', 'DLS_L_Acc_norm_mean',
            'SC_Acc_norm_range', 'DLS_R_Acc_norm_range', 'DLS_L_Acc_norm_range',
            'SC_Acc_norm_rms', 'DLS_R_Acc_norm_rms', 'DLS_L_Acc_norm_rms',
            'SC_Acc_norm_std', 'DLS_R_Acc_norm_std', 'DLS_L_Acc_norm_std',
            'SC_Acc_norm_skew', 'DLS_R_Acc_norm_skew', 'DLS_L_Acc_norm_skew',
            'SC_Acc_norm_kurtosis', 'DLS_R_Acc_norm_kurtosis', 'DLS_L_Acc_norm_kurtosis',
            'SC_Acc_norm_SamEn', 'DLS_R_Acc_norm_SamEn', 'DLS_L_Acc_norm_SamEn',
            'SC_Acc_norm_DAmp', 'DLS_R_Acc_norm_DAmp', 'DLS_L_Acc_norm_DAmp',
            'SC_Acc_norm_DFreq', 'DLS_R_Acc_norm_DFreq', 'DLS_L_Acc_norm_DFreq'
                    ]]
        X = df_feature.values


        print(df)


        # To combine Moderate and Mild at Admission
        k = 0
        for i in df['Ambul_AD']:
            if i == 'Moderate':
                df['Ambul_AD'].values[k] = 'Mild'
            k+=1

        # To combine Moderate and Mild at Discharge
        k = 0
        for i in df['Ambul_DC']:
            if i == 'Moderate':
                df['Ambul_DC'].values[k] = 'Mild'

        #         print('ID', k+1, ', AD: ', df['Ambul_AD'].values[k], ', DC: ', df['Ambul_DC'].values[k])
            k+=1

        factor = pd.factorize(df['Ambul_DC'])
        df.Ambul_DC = factor[0]
        definitions = factor[1]
        #         print(df.Ambul_DC)

        # Normalize features
        min_max_scaler = preprocessing.MinMaxScaler()
        X_scale = min_max_scaler.fit_transform(X)
        y = df['Ambul_DC'].values

        # print(X_scale)
        # print(y)

        # Leave-one-subject-out cross validation
        loo = LeaveOneOut()
        loo.get_n_splits(X)

        i = 0
        y_test_ = np.zeros((len(y)))
        y_pred_ = np.zeros((len(y)))
        y_probs_ = np.zeros((len(y),2))


        for k in range(100):
            correct = 0
            incorrect = 0
            for train_index, test_index in loo.split(X):
                print("TRAIN:", train_index, "TEST:", test_index)
                X_train, X_test = X_scale[train_index], X_scale[test_index]
                y_train, y_test = y[train_index], y[test_index]


            #   # Machine learning models
                if ML_model == 0:
                    clf = BalancedBaggingClassifier(random_state=k)
                    filename = 'Random100_Eval_BBC_Demo_SD.csv'
                elif ML_model == 1:
                    clf = BalancedRandomForestClassifier(n_estimators=100, random_state=k, bootstrap=False)
                    filename = 'Random100_Eval_BRF_Demo_SD.csv'
                elif ML_model == 2:
                    clf = RUSBoostClassifier(random_state=k)
                    filename = 'Random100_Eval_RB_Demo_SD.csv'
                
                clf.fit(X_train,y_train)
                y_pred = clf.predict(X_test)

                y_test_[i] = y_test
                y_pred_[i] = y_pred
                y_probs_[i] = clf.predict_proba(X_test)

                reversefactor = dict(zip(range(2),definitions))
                y_test = np.vectorize(reversefactor.get)(y_test)
                y_pred = np.vectorize(reversefactor.get)(y_pred)  

    #             print(y_pred)
    #             print(y_test)

                if y_test == ['Mild'] and y_pred == ['Mild']:
                    CM[0][0] += 1
                elif y_test == ['Mild'] and y_pred == ['Severe']:
                    CM[0][1] += 1
                elif y_test == ['Severe'] and y_pred == ['Severe']:
                    CM[1][1] += 1
                elif y_test == ['Severe'] and y_pred == ['Mild']:
                    CM[1][0] += 1

    #             print(CM)


                if y_test == y_pred:
                    correct = correct + 1
                else:
                    incorrect = incorrect + 1

                i = i+1
                print("Iteration:", i)  

    #         print("Correct: ", correct)
    #         print("Incorrect: ", incorrect)

            target_names = ['Mild', 'Severe']    
            print(confusion_matrix(y_test_,y_pred_))
            print(classification_report(y_test_,y_pred_, target_names = target_names))
            print('CoT =',n,', random =',k, 'filename = ', filename)
            W_F1[CoT][k] = metrics.f1_score(y_test_, y_pred_, average='weighted')
            i = 0

        k = 0
        CoT += 1
    print(W_F1)
    print(np.shape(W_F1))

    Eval_save = pd.DataFrame({'CoT10': W_F1[0],'CoT20': W_F1[1],'CoT30': W_F1[2],'CoT60': W_F1[3],'CoT90': W_F1[4],'CoT120': W_F1[5],'CoT180': W_F1[6],'CoT240': W_F1[7],'CoT300': W_F1[8],'CoT360': W_F1[9]})
    print(Eval_save)

    SAVEPATH = Path(r"X:\Lab Members\Sung\SYSHIN0228\RERC_STARS_Sensor_Analysis\Prediction_Admission_AIR_AoM_Analysis\Prediction_Machine_Learning")
    Eval_save.to_csv(SAVEPATH/filename, index=False) 
    
    CoT = 0


    subject  Age  SexM0F1   Height  Weight type  timesinceCVA Lesion_location  \
56    CVA02   62        1  160.000   61.00    1             3               0   
58    CVA04   80        0  177.800  102.10    1             7               1   
59    CVA05   48        1  160.020  103.10    1             9               1   
60    CVA06   76        0  184.404   97.80    1             4               0   
63    CVA09   61        1  175.000  107.60    1            11               1   
64    CVA10   68        1  157.480   64.20    1             6               0   
65    CVA11   62        0  167.640   66.20    0             8               1   
66    CVA12   44        1  162.500   76.40    1            11               1   
67    CVA13   49        0  190.500   74.50    1            11               1   
68    CVA14   56        0  187.000   84.60    1             6               0   
69    CVA15   73        0  179.800   95.70    1            99               0   
70    CVA16   51        0  1

Iteration: 9
TRAIN: [ 0  1  2  3  4  5  6  7  8 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40] TEST: [9]
Iteration: 10
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40] TEST: [10]
Iteration: 11
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40] TEST: [11]
Iteration: 12
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40] TEST: [12]
Iteration: 13
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40] TEST: [13]
Iteration: 14
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40] TEST: [14]
Iteration: 15
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 16 17

Iteration: 28
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 29 30 31 32 33 34 35 36 37 38 39 40] TEST: [28]
Iteration: 29
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 30 31 32 33 34 35 36 37 38 39 40] TEST: [29]
Iteration: 30
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 31 32 33 34 35 36 37 38 39 40] TEST: [30]
Iteration: 31
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 32 33 34 35 36 37 38 39 40] TEST: [31]
Iteration: 32
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 33 34 35 36 37 38 39 40] TEST: [32]
Iteration: 33
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 34 35 36 37 38 39 40] TEST: [33]
Iteration: 34
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 

Iteration: 6
TRAIN: [ 0  1  2  3  4  5  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40] TEST: [6]
Iteration: 7
TRAIN: [ 0  1  2  3  4  5  6  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40] TEST: [7]
Iteration: 8
TRAIN: [ 0  1  2  3  4  5  6  7  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40] TEST: [8]
Iteration: 9
TRAIN: [ 0  1  2  3  4  5  6  7  8 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40] TEST: [9]
Iteration: 10
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40] TEST: [10]
Iteration: 11
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40] TEST: [11]
Iteration: 12
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 13 14 15 16 17 18 19

Iteration: 18
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40] TEST: [18]
Iteration: 19
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40] TEST: [19]
Iteration: 20
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40] TEST: [20]
Iteration: 21
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40] TEST: [21]
Iteration: 22
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40] TEST: [22]
Iteration: 23
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40] TEST: [23]
Iteration: 24
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 

...

Iteration: 8
TRAIN: [ 0  1  2  3  4  5  6  7  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40] TEST: [8]
Iteration: 9
TRAIN: [ 0  1  2  3  4  5  6  7  8 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40] TEST: [9]
Iteration: 10
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40] TEST: [10]
Iteration: 11
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40] TEST: [11]
Iteration: 12
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40] TEST: [12]
Iteration: 13
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40] TEST: [13]
Iteration: 14
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 15 16 17 1

Iteration: 19
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40] TEST: [19]
Iteration: 20
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40] TEST: [20]
Iteration: 21
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40] TEST: [21]
Iteration: 22
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40] TEST: [22]
Iteration: 23
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40] TEST: [23]
Iteration: 24
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40] TEST: [24]
Iteration: 25
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 

Iteration: 30
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 31 32 33 34 35 36 37 38 39 40] TEST: [30]
Iteration: 31
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 32 33 34 35 36 37 38 39 40] TEST: [31]
Iteration: 32
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 33 34 35 36 37 38 39 40] TEST: [32]
Iteration: 33
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 34 35 36 37 38 39 40] TEST: [33]
Iteration: 34
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 35 36 37 38 39 40] TEST: [34]
Iteration: 35
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 36 37 38 39 40] TEST: [35]
Iteration: 36
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 

Iteration: 41
[[25  6]
 [ 4  6]]
              precision    recall  f1-score   support

        Mild       0.86      0.81      0.83        31
      Severe       0.50      0.60      0.55        10

    accuracy                           0.76        41
   macro avg       0.68      0.70      0.69        41
weighted avg       0.77      0.76      0.76        41

CoT = 10 , random = 11 filename =  Random100_Eval_BBC_Demo_SD.csv
TRAIN: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40] TEST: [0]
Iteration: 1
TRAIN: [ 0  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40] TEST: [1]
Iteration: 2
TRAIN: [ 0  1  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40] TEST: [2]
Iteration: 3
TRAIN: [ 0  1  2  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 3

KeyboardInterrupt: 